## 환경 변수 설정

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

## Pinecone 클라이언트 초기화

- Pinecone 객체 생성

In [2]:
from pinecone import Pinecone

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

[
    {
        "name": "embedding-test",
        "metric": "cosine",
        "host": "embedding-test-znzg6hs.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "us-east-1",
                "cloud": "aws",
                "read_capacity": {
                    "mode": "OnDemand",
                    "status": {
                        "state": "Ready",
                        "current_shards": null,
                        "current_replicas": null
                    }
                }
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "wiki",
        "metric": "cosine",
        "host": "wiki-znzg6hs.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "region": "us-east-1",
         

## Index 객체 생성
- index 객체 : embedding vector 저장 tool
- 유사도 기반의 검색 수행

In [3]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "wiki"

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Tue, 30 Dec 2025 00:43:51 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '82',
                                    'x-pinecone-request-id': '7205665100241439972',
                                    'x-pinecone-request-latency-ms': '82',
                                    'x-pinecone-response-duration-ms': '83'}},
 'dimension': 1536,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki_ns1': {'vector_count': 1293}},
 'storageFullness': 0.0,
 'total_vector_count': 1293,
 'vector_type': 'dense'}

# 검색하기
- vector db를 통해 질문에 대한 유사도가 가장 높은 top-k 문서 검색

## 방법 1. pinecone API 사용하기

#### 질문을 vector로 encoding하기

In [17]:
# embedding 객체 생성

from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
)

In [6]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000018FD0619220>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000018FD0BF7E00>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [21]:
# 질문을 embedding vector로 변환
question = ["백남준에 대해 알려줘"]

In [22]:
embedded_question = embedding.embed_documents(question)

In [24]:
len(embedded_question[0])

1536

#### encoding한 질문을 사용하여, wiki index에서 질문과 가장 유사한 3개의 항목을 추출.

In [27]:
query_result = index.query(
    namespace="wiki_ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=3,  # 상위 3개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
)
print(query_result)

QueryResponse(matches=[{'id': '5f6af7bd-8f98-4dd8-ac30-53253f843add',
 'metadata': {'chunk_id': 0,
              'full_text': '백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 '
                           '태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).\n'
                           '\n'
                           '생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 '
                           '활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 '
                           "예술가로서 '비디오 아트의 창시자'로 알려져 있다.\n"
                           '\n'
                           '생애',
              'title': '백남준',
              'url': 'https://ko.wikipedia.org/wiki/%EB%B0%B1%EB%82%A8%EC%A4%80',
              'wiki_id': '38'},
 'score': 0.54477793,
 'values': []}, {'id': 'cd233db3-e4d2-453b-9dc4-cdb8503936c8',
 'metadata': {'chunk_id': 31,
              'full_text': '《청년, 백남준: 초기 예술의 융합 미학》\n'
                           '연구자 임산(현 동덕여대 큐레이터학과 교수) 씨의 연구서로서 단행본 출간.\n'
                           ' 《백남준을 말하다》\n

In [36]:
# query_result.matches
for row in query_result.matches:
    # full_text 추출하기 
    print(row.metadata['full_text'])
    print('-'*30)

백남준(白南準, 1932년 7월 20일 ~ 2006년 1월 29일, 서울 출생, )은 한국 태생의 세계적인 비디오 아트 예술가, 작곡가, 전위 예술가이다. 본관은 수원(水原).

생전에 뉴욕, 쾰른, 도쿄, 마이애미와 서울에 주로 거주한 그는 여러 가지 매체로 예술 활동을 하였다. 특히 비디오 아트라는 새로운 예술의 범주를 발전시켰다는 평가를 받는 예술가로서 '비디오 아트의 창시자'로 알려져 있다.

생애
------------------------------
《청년, 백남준: 초기 예술의 융합 미학》
연구자 임산(현 동덕여대 큐레이터학과 교수) 씨의 연구서로서 단행본 출간.
 《백남준을 말하다》
국내의 백남준 지인들이 각자 얽힌 인연을 바탕으로 나름의 해석을 담은 책 출간.
 박정진. 《(eBOOK)굿으로 보는 백남준 비디오아트 읽기》. 한국학술정보. 2014년.
------------------------------
관련 영상 
 백남준의 위성예술 1부 (1984년 방송)
 백남준의 위성예술 2부 (1984년 방송)
 백남준의 비디오아트세계 (1984년 방송)
 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집

각주

외부 링크 

  백남준아트센터
 ARTCYCLOPEDIA에서 백남준 항목
 백남준의 세계 (출처:Guggenheim.org)
 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)
 백남준 일대기 (출처:@ MedienKunstNetz)}
 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)
 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)
 백남준 공식 블로그
------------------------------


### 전체 코드 통합하기

In [2]:
## 코드 통합
question = ["김대중 대통령의 행적에 대해 알여줘"]

from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

#벡터 DB, index 객체 생성
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "wiki"
index = pc.Index(index_name)

# embedding 객체생성 및 질문 embedding

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
)

embedded_question = embedding.embed_documents(question)

# top3 검색
query_result = index.query(
    namespace="wiki_ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=3,  # 상위 3개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
)


# 결과 추출
for row in query_result.matches:
    # full_text 추출하기 
    print(row.metadata['full_text'])
    print('-'*30)

1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령
------------------------------
취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아 연방평의회 의장, 알렉산드르 로스큐프 외무차관, 리하르트 폰 바이츠제커 전 독일 대통령, 수파차이 파닛팍디 WHO 사무총장, 도널드 존스턴 OECD 사무총장이 초대되었으며
------------------------------
15대 대선을 앞두고 통합민주당(1995년)의 대통령 후보 조순이 신한국당의 이회창과 연대 및 합당을 결정하면서 통추 내에서는 격론이 벌어졌다. 이부영·이철 등은 "3김 정치를 청산해야 한다"라며 신한국당을 선택하자고 주장하였고, 노무현·김원기·김정길 등은 "군사정권과 그 후예들을 심판하여 50년 만의 정권교체를 이룩해야 한다"라며 새정치국민회의 입당을 주장하였다

In [ ]:
# 함수 호출
question = ["김대중 대통령의 행적에 대해 알여줘"]
top_k_context = search_top_k(question)

In [9]:
## 코드 통합


from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
import os

def search_top_k(question):

    load_dotenv(override=True)
    
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    
    #벡터 DB, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체생성 및 질문 embedding
    
    embedding = OpenAIEmbeddings(
        model = "text-embedding-3-small",
        api_key = OPENAI_API_KEY
    )
    
    embedded_question = embedding.embed_documents(question)
    
    # top3 검색
    query_result = index.query(
        namespace="wiki_ns1",  # wiki-ns1 네임스페이스를 지정합니다.
        vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
        top_k=3,  # 상위 3개의 결과를 반환합니다.
        include_vector=False,  # 벡터를 포함하지 않습니다.
        include_metadata=True  # 메타데이터를 포함합니다.
    )
    
    # results = []
    # # 결과 추출
    # for row in query_result.matches:
    #     # full_text 추출하기 
    #     # print(row.metadata['full_text'])
    #     results.append(row.metadata['full_text'])
    #     # results = results + f"\n{'-'*30}\n"
    #     # print('-'*30)
    results = ""
    # 결과 추출
    for row in query_result.matches:
        # full_text 추출하기 
        # print(row.metadata['full_text'])
        results = results + row.metadata['full_text']
        # results = results + f"\n{'-'*30}\n"
        # print('-'*30)
    
    return results

In [10]:
# 함수 호출
question = ["김대중 대통령의 행적에 대해 알여줘"]
top_k_context = search_top_k(question)
print(top_k_context)

1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아 연방평의회 의장, 알렉산드르 로스큐프 외무차관, 리하르트 폰 바이츠제커 전 독일 대통령, 수파차이 파닛팍디 WHO 사무총장, 도널드 존스턴 OECD 사무총장이 초대되었으며15대 대선을 앞두고 통합민주당(1995년)의 대통령 후보 조순이 신한국당의 이회창과 연대 및 합당을 결정하면서 통추 내에서는 격론이 벌어졌다. 이부영·이철 등은 "3김 정치를 청산해야 한다"라며 신한국당을 선택하자고 주장하였고, 노무현·김원기·김정길 등은 "군사정권과 그 후예들을 심판하여 50년 만의 정권교체를 이룩해야 한다"라며 새정치국민회의 입당을 주장하였다. 결국 1997년 11월 노무현은 김정길, 김원기 등의 집행위원들과 함께 새정치국민회의에 입당하여 김대중을 지지하

## 방법 2. langchain API 사용
- 질문에 대한 유사도 검색 수행

In [24]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

#벡터 DB, index 객체 생성


### index 객체 생성

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "wiki"
index = pc.Index(index_name)


### embedding 객체생성

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
)

### PineconeVectorStore 객체 생성

In [31]:
from langchain_pinecone import PineconeVectorStore

# pineconevectorstore 객체 생성
vector_store = PineconeVectorStore(index=index,        
                                   embedding=embedding, 
                                   text_key="full_text"   # text키 =db 저장text
                                  )

In [32]:
question = ["김대중 대통령의 행적에 대해 알여줘"]

results = vector_store.similarity_search(
    query=question[0],    # query는 텍스트 형식으로 입력
    k=3, 
    namespace="wiki_ns1")

In [33]:
for result in results:
    print(result)
    # print(result.metadata)
    # print(result.page_content)
    print("-"*30)

page_content='1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령' metadata={'chunk_id': 189, 'title': '노무현', 'url': 'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%AC%B4%ED%98%84', 'wiki_id': '85'}
------------------------------
page_content='취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아 연방평의회 의장, 알렉산드르 로스큐프 외무차관, 리하르트 폰 바이츠제커 전 독일 대통령, 수파차이 파닛팍디 WHO 사무총장, 도널드 존스턴 OECD 사무총장이 초대되었으며' metadata={'chunk_id': 40, 'title': '노무현', 'url': 'https://ko.wikipedia.org/wi

### 전체 코드 통합

In [40]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

def sm_search_top_k(question):
    load_dotenv(override=True)
    
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    
    #벡터 DB, index 객체 생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
    )
    
    # pineconevectorstore 객체 생성
    vector_store = PineconeVectorStore(index=index,        
                                   embedding=embedding, 
                                   text_key="full_text"
                                  )
    
    
    results = vector_store.similarity_search(
    query=question[0], # question 문자열을 embedding vector로 자동 변환 됨.
    k=3, 
    namespace="wiki_ns1")

    context = ""
    for result in results:
        context = context + result.page_content + f"\n{'-'*30}\n"
    # print(result.metadata)
    # print(result.page_content)
    # print("-"*30)
    
    return context

In [41]:
question = ["김대중 대통령의 행적에 대해 알여줘"]
top_k_context = sm_search_top_k(question)
print(top_k_context)

1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령
------------------------------
취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아 연방평의회 의장, 알렉산드르 로스큐프 외무차관, 리하르트 폰 바이츠제커 전 독일 대통령, 수파차이 파닛팍디 WHO 사무총장, 도널드 존스턴 OECD 사무총장이 초대되었으며
------------------------------
15대 대선을 앞두고 통합민주당(1995년)의 대통령 후보 조순이 신한국당의 이회창과 연대 및 합당을 결정하면서 통추 내에서는 격론이 벌어졌다. 이부영·이철 등은 "3김 정치를 청산해야 한다"라며 신한국당을 선택하자고 주장하였고, 노무현·김원기·김정길 등은 "군사정권과 그 후예들을 심판하여 50년 만의 정권교체를 이룩해야 한다"라며 새정치국민회의 입당을 주장하였다